In [2]:
!nvidia-smi

Thu May 20 07:51:33 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 418.67       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla V100-PCIE...  On   | 00000000:00:05.0 Off |                  Off |
| N/A   33C    P0    27W / 250W |      0MiB / 32480MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [3]:
import os, time
import mmcv
from mmcv import Config
from mmdet.datasets import (build_dataloader, build_dataset, replace_ImageToTensor) # 앞의 2개 --> mmdet/datasets/builder.py에 존재 / 마지막 1개 --> 
from mmdet.models import build_detector #mmdet/models/builder.py에 존재
from mmdet.apis import train_detector, single_gpu_test # mmdet/apis/train.py, test.py에 존재
from mmcv.runner import load_checkpoint
from mmcv.parallel import MMDataParallel
import pandas as pd
from pandas import DataFrame
from pycocotools.coco import COCO
import numpy as np
import json, wandb

## train part

In [4]:
classes = ("UNKNOWN", "General trash", "Paper", "Paper pack", "Metal", "Glass", 
           "Plastic", "Styrofoam", "Plastic bag", "Battery", "Clothing")
# config file 들고오기
#cfg = Config.fromfile('../configs/faster_rcnn/faster_rcnn_r50_fpn_1x_coco.py')
#파일 교체 시 여기 이름 바꾸기
config_dir_name = 'hrnet'
config_file_name = 'faster_rcnn_hrnetv2p_w40_2x_coco'
cfg = Config.fromfile('../configs/'+config_dir_name+'/'+config_file_name+'.py')

PREFIX = '/opt/ml/input/data/'

# dataset 바꾸기
cfg.data.train.classes = classes
cfg.data.train.img_prefix = PREFIX
cfg.data.train.ann_file = PREFIX + 'train+pseudo.json'
cfg.data.train.pipeline[2]['img_scale'] = (512, 512)

cfg.data.val.classes = classes
cfg.data.val.img_prefix = PREFIX
cfg.data.val.ann_file = PREFIX + 'val.json'
cfg.data.val.pipeline[1]['img_scale'] = (512, 512)

cfg.data.test.classes = classes
cfg.data.test.img_prefix = PREFIX
cfg.data.test.ann_file = PREFIX + 'test.json'
cfg.data.test.pipeline[1]['img_scale'] = (512, 512)

cfg.data.samples_per_gpu = 16    #batch size

cfg.seed=42 #42로 고정
cfg.gpu_ids = [0]

cfg.model.roi_head.bbox_head.num_classes = 11

cfg.optimizer_config.grad_clip = dict(max_norm=35, norm_type=2)

#working directory 이름 설정
working_dir_name = '../work_dirs/' + config_file_name+'_psudoLabeling' +'_'+str(cfg.runner.max_epochs)
cfg.work_dir = working_dir_name


## wandb setting

In [5]:
group_name = 'hrnet'; project_name = 'gjtak'; run_name = config_file_name
config_list = {
    'epoch' : cfg.runner.max_epochs,
    'batch_size' :  cfg.data.samples_per_gpu,
    'optimizer' : cfg.optimizer,
    'optimizer_config' : cfg.optimizer_config,
    'lr_config' : cfg.lr_config
}
cfg.log_config.hooks[1].init_kwargs['group']=group_name # group name(option)
cfg.log_config.hooks[1].init_kwargs['name'] = run_name # run name
cfg.log_config.hooks[1].init_kwargs['config'] = config_list # config

### config 저장

In [6]:
# config 객체 -> 파일(json)로 저장(저장 장소 = working directory와 같이)
#working directory 없으면 생성하기
try:
    if not os.path.exists(working_dir_name):
        os.makedirs(working_dir_name)
except OSError :
    print('Error: Creating directory :: ' + working_dir_name)

#cfg 객체 -> string(str이용) -> dict(eval이용) -> json 으로 저장
cfg_str = str(cfg)[str(cfg).find(')')+3:]
cfg_dict = eval(cfg_str)
with open(working_dir_name+'/config.json','w') as config_json_file :
    json.dump(cfg_dict, config_json_file, indent="\t")


In [7]:
#config 저장 확인
with open(working_dir_name+'/config.json', 'r') as f :
    json_data = json.load(f)
#json_data

In [8]:
model = build_detector(cfg.model)

2021-05-20 07:51:43,106 - mmdet - INFO - load model from: open-mmlab://msra/hrnetv2_w40
2021-05-20 07:51:43,107 - mmdet - INFO - Use load_from_openmmlab loader
2021-05-20 07:51:43,927 - mmdet - WARNING - The model and loaded state dict do not match exactly

unexpected key in source state_dict: incre_modules.0.0.conv1.weight, incre_modules.0.0.bn1.weight, incre_modules.0.0.bn1.bias, incre_modules.0.0.bn1.running_mean, incre_modules.0.0.bn1.running_var, incre_modules.0.0.bn1.num_batches_tracked, incre_modules.0.0.conv2.weight, incre_modules.0.0.bn2.weight, incre_modules.0.0.bn2.bias, incre_modules.0.0.bn2.running_mean, incre_modules.0.0.bn2.running_var, incre_modules.0.0.bn2.num_batches_tracked, incre_modules.0.0.conv3.weight, incre_modules.0.0.bn3.weight, incre_modules.0.0.bn3.bias, incre_modules.0.0.bn3.running_mean, incre_modules.0.0.bn3.running_var, incre_modules.0.0.bn3.num_batches_tracked, incre_modules.0.0.downsample.0.weight, incre_modules.0.0.downsample.1.weight, incre_modules.0

In [9]:
datasets = [build_dataset(cfg.data.train)]

loading annotations into memory...
Done (t=3.93s)
creating index...
index created!


In [10]:
start = time.time()  # 시작 시간 저장
train_detector(model, datasets[0], cfg, distributed=False, validate=True)
print("time(sec) :", time.time() - start)  # 현재시각 - 시작시간 = 실행 시간(단위 : 초)

loading annotations into memory...
2021-05-20 07:51:54,273 - mmdet - INFO - Start running, host: root@a0934ebc062a, work_dir: /opt/ml/p3-ims-obd-hansarang/gjtak_works/Object Detection/code/mmdetection_trash/work_dirs/faster_rcnn_hrnetv2p_w40_2x_coco_psudoLabeling_24
2021-05-20 07:51:54,273 - mmdet - INFO - workflow: [('train', 1)], max: 24 epochs
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
Done (t=0.89s)
creating index...
index created!
wandb: Currently logged in as: pstage3_det (use `wandb login --relogin` to force relogin)


pn_cls: 0.0117, loss_rpn_bbox: 0.0281, loss_cls: 0.1555, acc: 94.3213, loss_bbox: 0.1899, loss: 0.3853, grad_norm: 1.5561
2021-05-20 09:54:42,234 - mmdet - INFO - Epoch [22][40/207]	lr: 2.000e-03, eta: 0:13:09, time: 1.392, data_time: 0.019, memory: 19081, loss_rpn_cls: 0.0111, loss_rpn_bbox: 0.0265, loss_cls: 0.1615, acc: 94.3164, loss_bbox: 0.1859, loss: 0.3849, grad_norm: 1.5667
2021-05-20 09:54:56,206 - mmdet - INFO - Epoch [22][50/207]	lr: 2.000e-03, eta: 0:12:56, time: 1.397, data_time: 0.020, memory: 19081, loss_rpn_cls: 0.0102, loss_rpn_bbox: 0.0263, loss_cls: 0.1502, acc: 94.5776, loss_bbox: 0.1872, loss: 0.3740, grad_norm: 1.6314
2021-05-20 09:55:10,123 - mmdet - INFO - Epoch [22][60/207]	lr: 2.000e-03, eta: 0:12:42, time: 1.392, data_time: 0.020, memory: 19081, loss_rpn_cls: 0.0121, loss_rpn_bbox: 0.0274, loss_cls: 0.1504, acc: 94.5947, loss_bbox: 0.1824, loss: 0.3724, grad_norm: 1.5608
2021-05-20 09:55:23,982 - mmdet - INFO - Epoch [22][70/207]	lr: 2.000e-03, eta: 0:12:29, 

train/loss_rpn_cls,0.00835
train/loss_rpn_bbox,0.02677
train/loss_cls,0.13196
train/acc,95.30151
train/loss_bbox,0.16578
train/loss,0.33286
train/grad_norm,1.37297
learning_rate,0.0002
momentum,0.9
_runtime,8341
_timestamp,1621505455


train/loss_rpn_cls,█▃▃▃▂▂▂▂▂▂▂▂▂▂▂▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train/loss_rpn_bbox,███▇▆▆▄▅▅▇▄▄▆▄▅▃▅▄▃▃▃▄▃▃▃▄▃▂▂▂▁▃▂▂▂▂▂▂▂▂
train/loss_cls,█▆▆▆▅▆▅▅▅▅▅▅▅▄▄▄▄▄▄▄▄▄▃▃▃▃▃▂▂▂▁▂▂▁▂▂▂▁▁▁
train/acc,▆▂▂▁▂▁▂▂▂▁▂▂▁▃▂▄▃▂▃▃▃▃▄▄▅▄▄▆▆▆▇▆▇▇▇▇▆█▇█
train/loss_bbox,▂███▆▇▅▆▅▆▅▅▆▄▅▄▅▅▅▄▄▄▃▃▂▄▄▂▃▂▁▃▁▂▂▂▂▁▂▁
train/loss,█▆▆▆▅▅▄▄▄▅▄▄▄▃▄▃▃▃▃▃▃▃▃▃▂▃▃▂▂▂▁▂▁▁▂▁▂▁▁▁
train/grad_norm,▇█▅▇▆▅▄▅▅▅▆▅▇▄▅▃▅▅█▆▆▅▅▅▅▆▆▂▂▂▂▃▂▂▂▃▃▂▁▁
learning_rate,▁▃▅▇███████████████████████▂▂▂▂▂▂▂▂▂▁▁▁▁
momentum,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
_runtime,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
_timestamp,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███


time(sec) : 8354.730295419693


## Inference part

In [14]:
epoch = cfg.runner.max_epochs
cfg.model.train_cfg = None

# checkpoint path
checkpoint_path = os.path.join(cfg.work_dir, f'epoch_{epoch}.pth')

In [15]:
dataset = build_dataset(cfg.data.test)
data_loader = build_dataloader(
        dataset,
        samples_per_gpu=1,
        workers_per_gpu=cfg.data.workers_per_gpu,
        dist=False,
        shuffle=False)

loading annotations into memory...
Done (t=0.01s)
creating index...
index created!


In [16]:
model = build_detector(cfg.model, test_cfg=cfg.get('test_cfg'))
checkpoint = load_checkpoint(model, checkpoint_path, map_location='cpu')

model.CLASSES = dataset.CLASSES
model = MMDataParallel(model.cuda(), device_ids=[0])

2021-05-20 10:33:43,168 - mmdet - INFO - load model from: open-mmlab://msra/hrnetv2_w40
2021-05-20 10:33:43,169 - mmdet - INFO - Use load_from_openmmlab loader
2021-05-20 10:33:43,690 - mmdet - WARNING - The model and loaded state dict do not match exactly

unexpected key in source state_dict: incre_modules.0.0.conv1.weight, incre_modules.0.0.bn1.weight, incre_modules.0.0.bn1.bias, incre_modules.0.0.bn1.running_mean, incre_modules.0.0.bn1.running_var, incre_modules.0.0.bn1.num_batches_tracked, incre_modules.0.0.conv2.weight, incre_modules.0.0.bn2.weight, incre_modules.0.0.bn2.bias, incre_modules.0.0.bn2.running_mean, incre_modules.0.0.bn2.running_var, incre_modules.0.0.bn2.num_batches_tracked, incre_modules.0.0.conv3.weight, incre_modules.0.0.bn3.weight, incre_modules.0.0.bn3.bias, incre_modules.0.0.bn3.running_mean, incre_modules.0.0.bn3.running_var, incre_modules.0.0.bn3.num_batches_tracked, incre_modules.0.0.downsample.0.weight, incre_modules.0.0.downsample.1.weight, incre_modules.0

In [17]:
output = single_gpu_test(model, data_loader, show_score_thr=0.05)

[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 837/837, 14.4 task/s, elapsed: 58s, ETA:     0s

In [18]:
prediction_strings = []
file_names = []
coco = COCO(cfg.data.test.ann_file)
imag_ids = coco.getImgIds()

class_num = 11
for i, out in enumerate(output):
    prediction_string = ''
    image_info = coco.loadImgs(coco.getImgIds(imgIds=i))[0]
    for j in range(class_num):
        for o in out[j]:
            prediction_string += str(j) + ' ' + str(o[4]) + ' ' + str(o[0]) + ' ' + str(o[1]) + ' ' + str(
                o[2]) + ' ' + str(o[3]) + ' '
        
    prediction_strings.append(prediction_string)
    file_names.append(image_info['file_name'])


submission = pd.DataFrame()
submission['PredictionString'] = prediction_strings
submission['image_id'] = file_names

submission.to_csv(os.path.join(cfg.work_dir, 'submission_'+config_file_name+'_psudoLabeling' +'_'+f'_{epoch}.csv'), index=None)
submission.head()

loading annotations into memory...
Done (t=0.01s)
creating index...
index created!


,PredictionString,image_id
0,8 0.9433308 173.66113 232.71133 464.47498 472....,batch_01_vt/0021.jpg
1,1 0.062591285 7.3584595 242.80472 283.3836 460...,batch_01_vt/0028.jpg
2,1 0.9233929 62.195374 378.62085 374.7003 511.4...,batch_01_vt/0031.jpg
3,8 0.88586533 46.39122 115.29739 381.84613 373....,batch_01_vt/0032.jpg
4,1 0.4257798 395.00494 276.52463 465.91333 342....,batch_01_vt/0070.jpg
